In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data=pd.read_csv('/content/data_final preprocessed_cdms.csv')
data

,cat__Varicella_1,cat__Varicella_2,cat__Mono or polysymptomatic_2,cat__Oligoclonal bands_0,cat__Oligoclonal bands_1,cat__Oligoclonal bands_2,cat__initial EDSS_1.0,cat__initial EDSS_2.0,cat__initial symptom_1.0,cat__initial symptom_2.0,...,cat__initial symptom_nan,cat__ final EDSS_1.0,cat__ final EDSS_2.0,remainder__Gender,remainder__LLSSEP,remainder__VEP,remainder__BAEP,remainder__Periventricular MRI,remainder__Infratentorial MRI,remainder__group
0,1,0,0,1,0,0,1,0,0,1,...,0,1,0,1,1,0,0,0,0,1
1,0,1,1,0,1,0,0,1,0,0,...,0,0,1,1,1,1,0,0,0,1
2,1,0,0,0,1,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
3,1,0,1,0,1,0,1,0,0,0,...,0,1,0,0,0,1,0,1,0,1
4,1,0,1,1,0,0,1,0,0,0,...,0,1,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,0,0,1,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
268,0,0,1,0,1,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,0
269,0,0,1,1,0,0,1,0,0,0,...,0,1,0,0,1,1,0,0,0,0
270,0,0,1,0,1,0,1,0,0,0,...,0,1,0,0,1,1,0,1,1,0


In [3]:
x=data.drop('remainder__group',axis=1)
y=data['remainder__group']

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=42)

In [5]:
x_train.shape

(217, 27)

In [7]:
!pip install --upgrade imbalanced-learn
from imblearn.over_sampling import BorderlineSMOTE
Borderline = BorderlineSMOTE(random_state=42)

x_train_res, y_train_res = Borderline.fit_resample(x_train, y_train)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.14.0
    Uninstalling imbalanced-learn-0.14.0:
      Successfully uninstalled imbalanced-learn-0.14.0


In [8]:
x_train_res.shape

(234, 27)

In [9]:
from sklearn.ensemble import RandomForestClassifier
opt=RandomForestClassifier(max_features=None, min_samples_leaf=2,
                       min_samples_split=5, n_estimators=300, random_state=42)
opt.fit(x_train_res,y_train_res)
y_pred = opt.predict(x_test)
y_prob = opt.predict_proba(x_test)[:, 1]

In [10]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score,accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))
print("PR AUC (average precision):", average_precision_score(y_test, y_prob))

Accuracy: 0.8909090909090909
              precision    recall  f1-score   support

           0       0.93      0.87      0.90        30
           1       0.85      0.92      0.88        25

    accuracy                           0.89        55
   macro avg       0.89      0.89      0.89        55
weighted avg       0.89      0.89      0.89        55

Confusion matrix:
 [[26  4]
 [ 2 23]]
ROC AUC: 0.9493333333333333
PR AUC (average precision): 0.9475491379540864


In [11]:
!pip install --upgrade numpy
!pip install --force-reinstall --no-cache-dir alibi
# !pip install --upgrade scikit-learn pandas numba


  Using cached numpy-2.4.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)
Using cached numpy-2.4.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 480, in run
    installed_dist = env.get_distribution(item)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3

In [13]:
!pip install alibi

def predict_fn(Z):
    return opt.predict(Z)

from alibi.explainers import AnchorTabular

feature_names = opt.feature_names_in_

explainer = AnchorTabular(
    predictor=predict_fn,
    feature_names=feature_names
)

explainer.fit(x_train_res.values)

idx = 0
explanation = explainer.explain(
    x_train_res.values[idx],
    threshold=0.95
)

print(explanation.anchor)

print("Precision:", explanation.precision)
print("Coverage:", explanation.coverage)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local

Anchor rules:
['remainder__Periventricular MRI <= 0.00', 'cat__initial symptom_2.0 > 0.00', 'cat__Mono or polysymptomatic_2 <= 0.00']

Precision: 1.0
Coverage: 0.0439
